Code to accompany Machine Learning Recipes #8. We'll write a Decision Tree Classifier, in pure Python. Below each of the methods, I've written a little demo to help explain what it does.

In [198]:
#fonction qui nous permettra d'afficher notre arbre de decision
from __future__ import print_function
import numpy as np
import pandas as pd

In [199]:
# Toy dataset.

training_data = pd.read_csv("data.csv")

### Visualisez les donnees (valeurs reelles) ###

In [200]:
training_data

,Color,Size,Season,Label
0,Red,1,Summer,Grape
1,Green,3,Spring,Apple
2,Red,3,Winter,Orange
3,Green,2,Winter,Kiwi
4,Yellow,3,Winter,Lemon
5,Green,2,Winter,Kiwi
6,Yellow,3,Winter,Lemon
7,Green,1,Summer,Grape
8,Red,3,Spring,Apple
9,Red,3,Winter,Orange


In [201]:
# Definissez X et y 

In [202]:
X = training_data.values[:,:3]
print(type(X))
y = np.array(training_data.Label)
#y=y.reshape(100,1)
print(type(y))
print(y.shape)


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(100,)


In [203]:
# Nous devons maintenant stocker les differents titres de categrories pour pouvoir afficher notre arbre
header = ["color", "size", "season", "label"]

In [204]:
# Definir la fonction permettant de trouver les differentes valeurs possibles pour chaque categorie #

In [205]:
def unique_vals(rows, col):
    return set([row[col] for row in rows])

In [206]:
# Testez votre fonction pour l'attribut "Seasons" ; vous devriez obtenir 'Spring, Summer, Winter)

In [207]:
unique_vals(X, 2)

{'Spring', 'Summer', 'Winter'}

In [208]:
# Definir la fonction permettant de compter chaque nombre de fruits dans le dataset (creer un dictionnaire)

In [209]:
def class_counts(rows_y):
    counts = {}  # a dictionary of label -> count.
    for row_y in rows_y:
        # in our dataset format, the label is always the last column
        label = row_y[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [210]:
## Lancez votre fonction
class_counts(y)
apple = ['Apple', 'Apple']
class_counts(apple)
## Elle devrait retourner : {'Grape': 19, 'Apple': 19, 'Orange': 20, 'Kiwi': 22, 'Lemon': 20} 

{'e': 2}

In [211]:
def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

In [212]:
#######
# Demo:
is_numeric(7)
is_numeric("Red")
#######

False

In [213]:
class Question:
    """A Question is used to partition a dataset.

    This class just records a 'column number' (e.g., 0 for Color) and a
    'column value' (e.g., Green). The 'match' method is used to compare
    the feature value in an example to the feature value stored in the
    question. See the demo below.
    """

    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

In [214]:
#######
# Demo:
# Let's write a question for a numeric attribute
Question(0, 2)

Is color >= 2?

In [215]:
# How about one for a categorical attribute
q = Question(0, 'Green')
q

Is color == Green?

In [216]:
# Let's pick an example from the training set...
example = X[1]
# ... and see if it matches the question
q.match(example) # this will be true, since the first example is Green.
#######

True

In [217]:
def partition(rows, question):
    """Partitions a dataset.

    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [218]:
#######
# Demo:
# Let's partition the training data based on whether rows are Red.
true_rows, false_rows = partition(X, Question(0, 'Red'))
# This will contain all the 'Red' rows.
print(false_rows[:8])

[array(['Green', 3, 'Spring'], dtype=object), array(['Green', 2, 'Winter'], dtype=object), array(['Yellow', 3, 'Winter'], dtype=object), array(['Green', 2, 'Winter'], dtype=object), array(['Yellow', 3, 'Winter'], dtype=object), array(['Green', 1, 'Summer'], dtype=object), array(['Green', 3, 'Winter'], dtype=object), array(['Yellow', 2, 'Winter'], dtype=object)]


In [219]:
# This will contain everything else.
false_rows
#######

[array(['Green', 3, 'Spring'], dtype=object),
 array(['Green', 2, 'Winter'], dtype=object),
 array(['Yellow', 3, 'Winter'], dtype=object),
 array(['Green', 2, 'Winter'], dtype=object),
 array(['Yellow', 3, 'Winter'], dtype=object),
 array(['Green', 1, 'Summer'], dtype=object),
 array(['Green', 3, 'Winter'], dtype=object),
 array(['Yellow', 2, 'Winter'], dtype=object),
 array(['Green', 3, 'Spring'], dtype=object),
 array(['Green', 2, 'Winter'], dtype=object),
 array(['Yellow', 2, 'Winter'], dtype=object),
 array(['Green', 3, 'Spring'], dtype=object),
 array(['Green', 3, 'Winter'], dtype=object),
 array(['Yellow', 3, 'Winter'], dtype=object),
 array(['Green', 3, 'Spring'], dtype=object),
 array(['Green', 2, 'Winter'], dtype=object),
 array(['Yellow', 3, 'Winter'], dtype=object),
 array(['Green', 3, 'Spring'], dtype=object),
 array(['Green', 2, 'Winter'], dtype=object),
 array(['Yellow', 3, 'Winter'], dtype=object),
 array(['Green', 3, 'Spring'], dtype=object),
 array(['Green', 2, 'Winter

In [220]:
def gini(rows):
    """Calculate the Gini Impurity for a list of rows.

    There are a few different ways to do this, I thought this one was
    the most concise. See:
    https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
    """
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [221]:
#######
# Demo:
# Let's look at some example to understand how Gini Impurity works.
#
# First, we'll look at a dataset with no mixing.
no_mixing = ['Apple','Apple']
# this will return 0
gini(y)

0.5751999999999999

In [222]:
# Now, we'll look at dataset with a 50:50 apples:oranges ratio
some_mixing = ['Apple','Orange']
# this will return 0.5 - meaning, there's a 50% chance of misclassifying
# a random example we draw from the dataset.
gini(some_mixing)

0.0

In [223]:
# Now, we'll look at a dataset with many different labels
lots_of_mixing = ['Apple',
                  'Orange',
                  'Grape',
                  'Grapefruit',
                  'Blueberry']
# This will return 0.8
gini(lots_of_mixing)
#######

0.5599999999999999

In [224]:
def info_gain(left, right, current_uncertainty):
    """Information Gain.

    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [225]:
#######
# Demo:
# Calculate the uncertainy of our training data.
current_uncertainty = gini(X)
current_uncertainty


0.5433999999999999

In [226]:
# How much information do we gain by partioning on 'Green'?
true_rows, false_rows = partition(X, Question(0, 'Green'))
info_gain(true_rows, false_rows, current_uncertainty)

0.06698825961140953

In [227]:
# What about if we partioned on 'Red' instead?
true_rows, false_rows = partition(X, Question(0,'Red'))
info_gain(true_rows, false_rows, current_uncertainty)

0.0717564522908784

In [228]:
# It looks like we learned more using 'Red' (0.37), than 'Green' (0.14).
# Why? Look at the different splits that result, and see which one
# looks more 'unmixed' to you.
#print(true_rows)
true_rows, false_rows = partition(X, Question(0,'Red'))
print(type(training_data))
print(type(X))
# Here, the true_rows contain only 'Grapes'.
print(true_rows[:3])
print(false_rows[:3])

<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>
[array(['Red', 1, 'Summer'], dtype=object), array(['Red', 3, 'Winter'], dtype=object), array(['Red', 3, 'Spring'], dtype=object)]
[array(['Green', 3, 'Spring'], dtype=object), array(['Green', 2, 'Winter'], dtype=object), array(['Yellow', 3, 'Winter'], dtype=object)]


In [229]:
# And the false rows contain two types of fruit. Not too bad.
false_rows

[array(['Green', 3, 'Spring'], dtype=object),
 array(['Green', 2, 'Winter'], dtype=object),
 array(['Yellow', 3, 'Winter'], dtype=object),
 array(['Green', 2, 'Winter'], dtype=object),
 array(['Yellow', 3, 'Winter'], dtype=object),
 array(['Green', 1, 'Summer'], dtype=object),
 array(['Green', 3, 'Winter'], dtype=object),
 array(['Yellow', 2, 'Winter'], dtype=object),
 array(['Green', 3, 'Spring'], dtype=object),
 array(['Green', 2, 'Winter'], dtype=object),
 array(['Yellow', 2, 'Winter'], dtype=object),
 array(['Green', 3, 'Spring'], dtype=object),
 array(['Green', 3, 'Winter'], dtype=object),
 array(['Yellow', 3, 'Winter'], dtype=object),
 array(['Green', 3, 'Spring'], dtype=object),
 array(['Green', 2, 'Winter'], dtype=object),
 array(['Yellow', 3, 'Winter'], dtype=object),
 array(['Green', 3, 'Spring'], dtype=object),
 array(['Green', 2, 'Winter'], dtype=object),
 array(['Yellow', 3, 'Winter'], dtype=object),
 array(['Green', 3, 'Spring'], dtype=object),
 array(['Green', 2, 'Winter

In [230]:
# On the other hand, partitioning by Green doesn't help so much.
true_rows, false_rows = partition(X, Question(0,'Green'))

# We've isolated one apple in the true rows.
true_rows

[array(['Green', 3, 'Spring'], dtype=object),
 array(['Green', 2, 'Winter'], dtype=object),
 array(['Green', 2, 'Winter'], dtype=object),
 array(['Green', 1, 'Summer'], dtype=object),
 array(['Green', 3, 'Winter'], dtype=object),
 array(['Green', 3, 'Spring'], dtype=object),
 array(['Green', 2, 'Winter'], dtype=object),
 array(['Green', 3, 'Spring'], dtype=object),
 array(['Green', 3, 'Winter'], dtype=object),
 array(['Green', 3, 'Spring'], dtype=object),
 array(['Green', 2, 'Winter'], dtype=object),
 array(['Green', 3, 'Spring'], dtype=object),
 array(['Green', 2, 'Winter'], dtype=object),
 array(['Green', 3, 'Spring'], dtype=object),
 array(['Green', 2, 'Winter'], dtype=object),
 array(['Green', 3, 'Spring'], dtype=object),
 array(['Green', 2, 'Winter'], dtype=object),
 array(['Green', 3, 'Spring'], dtype=object),
 array(['Green', 2, 'Winter'], dtype=object),
 array(['Green', 3, 'Spring'], dtype=object),
 array(['Green', 2, 'Winter'], dtype=object),
 array(['Green', 3, 'Spring'], dty

In [231]:
# But, the false-rows are badly mixed up.
false_rows
#######

[array(['Red', 1, 'Summer'], dtype=object),
 array(['Red', 3, 'Winter'], dtype=object),
 array(['Yellow', 3, 'Winter'], dtype=object),
 array(['Yellow', 3, 'Winter'], dtype=object),
 array(['Red', 3, 'Spring'], dtype=object),
 array(['Red', 3, 'Winter'], dtype=object),
 array(['Yellow', 2, 'Winter'], dtype=object),
 array(['Red', 1, 'Summer'], dtype=object),
 array(['Red', 3, 'Winter'], dtype=object),
 array(['Yellow', 2, 'Winter'], dtype=object),
 array(['Red', 1, 'Summer'], dtype=object),
 array(['Red', 3, 'Winter'], dtype=object),
 array(['Yellow', 3, 'Winter'], dtype=object),
 array(['Red', 1, 'Summer'], dtype=object),
 array(['Red', 3, 'Winter'], dtype=object),
 array(['Yellow', 3, 'Winter'], dtype=object),
 array(['Red', 1, 'Summer'], dtype=object),
 array(['Red', 3, 'Winter'], dtype=object),
 array(['Yellow', 3, 'Winter'], dtype=object),
 array(['Red', 1, 'Summer'], dtype=object),
 array(['Red', 3, 'Winter'], dtype=object),
 array(['Yellow', 3, 'Winter'], dtype=object),
 array([

In [232]:
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # number of columns

    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value

            question = Question(col, val)

            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [233]:
#######
# Demo:
# Find the best question to ask first for our toy dataset.
best_gain, best_question = find_best_split(X)
best_question
# FYI: is color == Red is just as good. See the note in the code above
# where I used '>='.
#######

Is size >= 2?

In [234]:
class Leaf:
    """A Leaf node classifies data.

    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """

    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [235]:
class Decision_Node:
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [236]:
def build_tree(rows):
    """Builds the tree.

    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """

    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gain, question = find_best_split(rows)

    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)

In [237]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [238]:
my_tree = build_tree(X)

In [239]:
print_tree(my_tree)

Is size >= 2?
--> True:
  Is color == Green?
  --> True:
    Is size >= 3?
    --> True:
      Predict {'Spring': 18, 'Winter': 2}
    --> False:
      Predict {'Winter': 20}
  --> False:
    Is color == Red?
    --> True:
      Predict {'Winter': 20, 'Spring': 1}
    --> False:
      Predict {'Winter': 20}
--> False:
  Predict {'Summer': 19}


In [240]:
def classify(row, node):
    """See the 'rules of recursion' above."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [241]:
#######
# Demo:
# The tree predicts the 1st row of our
# training data is an apple with confidence 1.
classify(y[0], my_tree)
#######

{'Summer': 19}

In [242]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [243]:
#######
# Demo:
# Printing that a bit nicer
print_leaf(classify(X[0], my_tree))
#######

{'Summer': '100%'}

In [244]:
#######
# Demo:
# On the second example, the confidence is lower
print_leaf(classify(X[1], my_tree))
#######

{'Spring': '90%', 'Winter': '10%'}

In [245]:
# Evaluate
testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

In [246]:
for row in testing_data:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))

Actual: Apple. Predicted: {'Spring': '90%', 'Winter': '10%'}
Actual: Apple. Predicted: {'Winter': '100%'}
Actual: Grape. Predicted: {'Winter': '95%', 'Spring': '4%'}
Actual: Grape. Predicted: {'Summer': '100%'}
Actual: Lemon. Predicted: {'Winter': '100%'}
